In [ ]:
pip install opencv-python pandas mediapipe numpy

# Collect sample situation image from Video

In [ ]:
import cv2


In [ ]:
vedio_name = 'VEDIO/Nonlight.mp4' # Chage Input Vedio Path Here 

In [ ]:
vid = cv2.VideoCapture(vedio_name)
vid.set(3, 1280)
vid.set(4, 720)
while(True):
    ret, frame = vid.read()
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        out = cv2.imwrite('capture1.jpg', frame)
        break
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

# Sketch square Working space (Packing) 

In [ ]:
cap = cv2.imread('capture1.jpg',cv2.IMREAD_COLOR)
x_w,y_w,w_w,h_w = cv2.selectROI(cap)

In [ ]:
img = cv2.imread('capture1.jpg')
starting_coordinate = (x_w,y_w)
ending_coordinate = (w_w+x_w,h_w+y_w)
color = (255,0,0)
new_image = cv2.rectangle(img, starting_coordinate , ending_coordinate, color)

cv2.imshow('image', img)

cv2.waitKey(0)

out_2 = cv2.imwrite('capture2.jpg', new_image)

print(f'{x_w,y_w,w_w,h_w}')

cv2.destroyAllWindows()

# Sketch square Drop-off  (Product placement)

In [ ]:
cap = cv2.imread('capture2.jpg',cv2.IMREAD_COLOR)
x_o,y_o,w_o,h_o = cv2.selectROI(cap)

In [ ]:
img = cv2.imread('capture2.jpg')
starting_coordinate = (x_o,y_o)
ending_coordinate = (w_o+x_o,h_o+y_o)
color = (0,0,255)
new_image = cv2.rectangle(img, starting_coordinate , ending_coordinate, color)

cv2.imshow('image', img)

cv2.waitKey(0)

out_2 = cv2.imwrite('capture3.jpg', new_image)
print(f'{x_o,y_o,w_o,h_o}')

cv2.destroyAllWindows()

# Run Algorithm & MediaPipe Holistic

In [ ]:
# Put input coordinate here 

x_w,y_w,w_w,h_w = 58, 105, 554, 835

x_o,y_o,w_o,h_o = 1240, 90, 654, 910

vedio_name = 'VEDIO/Nonlight.mp4'

cycle = 10 # Cycle time


In [ ]:
%%time
import time
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
list_for_data=[]
first_station = []
second_station = []
pick = 0
drop = 1
cap = cv2.VideoCapture(vedio_name)
# Get the frame rate (frames per second) and number of frames
fps = cap.get(cv2.CAP_PROP_FPS)
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# Calculate the duration of the video in seconds
duration = num_frames / fps
# Initialize the timer
timer = 0
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, image = cap.read()
        # Increment the timer by the delay between frames
        timer += int(1000 / fps)
        # Calculate the current time in seconds
        current_time = timer / 1000
        if not success:
            print("Ignoring empty camera frame.")
                  # If loading a video, use 'break' instead of 'continue'.
            cv2.destroyAllWindows() 
            break
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        # Draw the pose annotation on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        lmlist = []
        if results.pose_landmarks:
            for ide, lm in enumerate(results.pose_landmarks.landmark):
                h,w,c = image.shape
                cx,cy = int(lm.x*w),int(lm.y*h)
                lmlist.append([ide,cx,cy])
                list_for_data.append([ide,cx,cy])
            # Flip the image horizontally for a selfie-view display.
        # Flip the image horizontally for a selfie-view display.
        cv2.putText(image, f"Time: {current_time:.2f}/{duration:.0f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        if len(lmlist)!=0:
            body = []
            for k in range(len(lmlist[0])):
                result = lmlist[11][k] + lmlist[12][k] + lmlist[23][k] + lmlist[24][k]
                body.append(result/4)
            if y_o+h_o >= body[2] >= y_o and x_o <= body[1] <=x_o+w_o:
                if drop == 0 :
                    second_station.append(current_time)
                    drop = 1
                    pick = 0
                    print(f'second_station:{current_time}')
            elif y_w+h_w >= body[2] >= y_w and x_w <= body[1] <=x_w+w_w:
                if pick == 0 :
                    first_station.append(current_time)
                    pick = 1
                    drop = 0
                    print(f'first_station:{current_time}')
                if len(first_station) == cycle+1 :
                    break
            #cv2.imshow('MediaPipe Pose',image)
            if cv2.waitKey(5) & 0xFF == ord('q'):
                cv2.destroyAllWindows() 
                break
        cv2.imshow('frame', image) 
cv2.destroyAllWindows() 
cap.release()

#### Transfrom data to Excel 
staion =first_station
cycle = []
for i in range(len(staion)-1):
    cycle.append(abs(staion[i+1]-staion[i]))
DATA_CYCLE = pd.DataFrame({'Cycle Time':cycle})
DATA_CYCLE.to_excel('Cycle_time.xlsx',index=False)